In [2]:
import pandas as pd
import os, pathlib
from scipy.stats import kendalltau
import numpy as np

### SciPy definiton of Kendall's Tau: 
### $$\frac{P - Q}{\sqrt{(P + Q + T) * (P + Q + U)}}$$
#### where P is the number of concordant pairs, Q the number of discordant pairs, T the number of ties only in x, and U the number of ties only in y. If a tie occurs for the same pair in both x and y, it is not added to either T or U. (Note that ties will have been broken randomly by the calc_rank function below.) 
#### See https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.kendalltau.html for more information.
#### Should we use non-negative definition of KT instead?

In [3]:
def calc_rank(seed, y):

    '''
    Function adapted from https://www.geeksforgeeks.org/rank-elements-array/
    Randomly breaks ties using np random seed
    
    Copied from utils.stabililty_utils.py
    '''

    # Set random seed
    np.random.seed(seed)

    # Initialize rank vector 
    R = [0 for i in range(len(y))] 

    # Create an auxiliary array of tuples 
    # Each tuple stores the data as well as its index in y 
    # T[][0] is the data and T[][1] is the index of data in y
    T = [(y[i], i) for i in range(len(y))] 
    
    # Sort T according to first element 
    T.sort(key=lambda x: x[0], reverse=True)

    # Loop through items in T
    i=0
    while i < len(y): 

        # Get number of elements with equal rank 
        j = i 
        while j < len(y) - 1 and T[j][0] == T[j + 1][0]: 
            j += 1
        n = j - i + 1

        # If there is no tie
        if n==1:
            
            # Get ID of this element
            idx = T[i][1] 
            
            # Set rank
            rank = i+1
            
            # Assign rank
            R[idx] = rank 
            
        # If there is a tie
        if n>1: 
            
            # Create array of ranks to be assigned
            ranks = list(np.arange(i+1, i+1+n)) 
            
            # Randomly shuffle the ranks
            np.random.shuffle(ranks) 
            
            # Create list of element IDs
            ids = [T[i+x][1] for x in range(n)] 
            
            # Assign rank to each element
            for ind, idx in enumerate(ids):
                R[idx] = ranks[ind] 

        # Increment i 
        i += n 
    
    # return rank vector
    return R

In [48]:
out_dir = pathlib.Path(os.getcwd()).parents[1] / 'out'

s_samples = 200
n_runs = 500

# Highest seed not yet used in data gen
seed = s_samples*((n_runs+1)*3+2)

# Initialize arrays to hold Kendall's Tau distances
noise_kts = np.zeros([s_samples, n_runs])
counter_kts_xres = np.zeros([s_samples, 2])
counter_kts_nonres = np.zeros([s_samples, 2])
kts_xres_a0_a1 = np.zeros(s_samples)
kts_nonres_a0_a1 = np.zeros(s_samples)

# Loop through s_samples
for s in range(1, s_samples+1):
    
    # Read in counterfactual data
    counter = pd.read_csv(out_dir/'counterfactual_data'/'stability'/'default'/'counter_samp_{}.csv'.format(s))
    
    # Get list of counterfactual Y columns
    counter_y_cols = [x for x in counter.columns if 'cf_y_' in x]
    
    # Calculate ranks for each counterfactual Y
    for y in counter_y_cols:
        counter['rank_'+y[5:]] = calc_rank(seed=seed, y=counter[y])
        seed += 1
    
    # Read in noise distribution data
    noise = pd.read_csv(out_dir/'synthetic_data'/'stability'/'default'/'rankings'/'observed_samp_{}.csv'.format(s))
    
    # Get original rank
    orig_rank = noise['rank']
    
    # Get KT distances between original rank and each rank from noise distribution
    # Average will give expected noise KT for this sample
    for n in range(1, n_runs+1):
        noise_kt, noise_p = kendalltau(orig_rank, noise['rank_{}'.format(n)])
        noise_kts[s-1][n-1] = noise_kt
    
    # For each intervention, A<-0 and A<-1
    for a in range(2):
        try:
            # Get KT distance between original rank and counterfactual Y with resolving X
            counter_kts_nonres[s-1][a] = kendalltau(orig_rank, counter['rank_nonres_a{}'.format(a)])[0]
            
            # Get KT distance between original rank and counterfactual Y with non-resolving X
            counter_kts_xres[s-1][a] = kendalltau(orig_rank, counter['rank_xres_a{}'.format(a)])[0]
            
        # Catch exception for A=a not present in original dataset
        # A<-a intervention will not have been performed
        except:
            counter_kts_nonres[s-1][a] = np.nan
            counter_kts_xres[s-1][a] = np.nan
    
    # Get KT distance between counterfactual ranks for intervention A<-0 and for intervention A<-1
    try:
        # Distance between ranks resulting from each intervention for non-resolving X
        kts_nonres_a0_a1[s-1] = kendalltau(counter['rank_nonres_a0'], counter['rank_nonres_a1'])[0]   
        
        # Distance between ranks resulting from each intervention for resolving X
        kts_xres_a0_a1[s-1] = kendalltau(counter['rank_xres_a0'], counter['rank_xres_a1'])[0]
        
    # Catch exception for only one value of A present in original dataset
    # Only one intervention will have been performed
    except:
        kts_nonres_a0_a1[s-1] = np.nan
        kts_xres_a0_a1[s-1] = np.nan

In [35]:
# Get expected KT distance between original rank and rank from re-sampled noise
# Expectation taken over n_runs of noise distribution
# E[ KT(original rank, rank from re-sampled noise) ]
# 1 value for each sample
exp_kt_noise = np.mean(noise_kts, axis=1)

# Get expected value of expected KT distance between original rank and rank from re-sampled noise
# Iterated expectation: expectation first taken over n_runs of noise distribution, then over s_samples
# E[ E[ KT(original rank, rank from re-sampled noise) ] ]
# 1 value for entire experiment trial
exp_exp_kt_noise = np.mean(exp_kt_noise)

In [36]:
# Get expected KT distance between original rank and counterfactual Y with non-resolving X
# For intervention A<-0
# Expectation taken over s_samples
# E[ KT(original rank, counterfactual rank with non-resolving X for A<-0) ]
# 1 value for entire experiment trial
exp_kt_counter_nonres_a0 = np.nanmean(counter_kts_nonres[:,0])

# Get expected KT distance between original rank and counterfactual Y with non-resolving X
# For intervention A<-1
# Expectation taken over s_samples
# E[ KT(original rank, counterfactual rank with non-resolving X for A<-1) ]
# 1 value for entire experiment trial
exp_kt_counter_nonres_a1 = np.nanmean(counter_kts_nonres[:,1])

# Get expected KT distance between original rank and counterfactual Y with resolving X
# For intervention A<-0
# Expectation taken over s_samples
# E[ KT(original rank, counterfactual rank with resolving X for A<-0) ]
# 1 value for entire experiment trial
exp_kt_counter_xres_a0 = np.nanmean(counter_kts_xres[:,0])

# Get expected KT distance between original rank and counterfactual Y with resolving X
# For intervention A<-1
# Expectation taken over s_samples
# E[ KT(original rank, counterfactual rank with resolving X for A<-1) ]
# 1 value for entire experiment trial
exp_kt_counter_xres_a1 = np.nanmean(counter_kts_xres[:,1])

$$\hat{E}\big[KT(\text{original rank, counterfactual rank with }\textbf{non-resolving X}\text{ for }\textbf{A$\leftarrow$0}) - \hat{E}[KT(\text{original rank, rank from re-sampled noise})]\big]$$

In [52]:
exp_kt_counter_nonres_a0 - exp_exp_kt_noise

0.7380536251402918

$$\hat{E}\big[KT(\text{original rank, counterfactual rank with }\textbf{non-resolving X}\text{ for }\textbf{A$\leftarrow$1}) - \hat{E}[KT(\text{original rank, rank from re-sampled noise})]\big]$$

In [51]:
exp_kt_counter_nonres_a1 - exp_exp_kt_noise

0.7400244444444444

$$\hat{E}\big[KT(\text{original rank, counterfactual rank with }\textbf{resolving X}\text{ for }\textbf{A$\leftarrow$0}) - \hat{E}[KT(\text{original rank, rank from re-sampled noise})]\big]$$

In [53]:
exp_kt_counter_xres_a0 - exp_exp_kt_noise

0.756684377104377

$$\hat{E}\big[KT(\text{original rank, counterfactual rank with }\textbf{resolving X}\text{ for }\textbf{A$\leftarrow$1}) - \hat{E}[KT(\text{original rank, rank from re-sampled noise})]\big]$$

In [54]:
exp_kt_counter_xres_a1 - exp_exp_kt_noise

0.7584688888888886

$$\hat{E}[KT(\text{counterfactual rank with }\textbf{non-resolving X}\text{ for }\textbf{A$\leftarrow$0}), \text{counterfactual rank with }\textbf{non-resolving X}\text{ for }\textbf{A$\leftarrow$1})]$$

In [55]:
exp_kt_nonres_a0_a1 = np.nanmean(kts_nonres_a0_a1)
exp_kt_nonres_a0_a1

0.9999999999999999

$$\hat{E}[KT(\text{counterfactual rank with }\textbf{resolving X}\text{ for }\textbf{A$\leftarrow$0}), \text{counterfactual rank with }\textbf{resolving X}\text{ for }\textbf{A$\leftarrow$1})]$$

In [56]:
exp_kt_xres_a0_a1 = np.nanmean(kts_xres_a0_a1)
exp_kt_xres_a0_a1

0.9999999999999999